In [3]:
# coding=UTF-8
from bs4 import BeautifulSoup 
from urllib.request import urlopen
import requests
import re
import time
from tqdm import tqdm
import random
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from pymorphy2 import MorphAnalyzer
from sklearn.metrics import accuracy_score

In [6]:
def writing_in_dict(soup):
    d_com = {}
    text_full = soup.find_all("div", attrs={"class": "responses__item__message markup-inside-small markup-inside-small--bullet"})
    regex = r"<.+>"
    comments_clean = []
    for each in text_full:
        result = re.sub(regex, '', str(each), 0, re.MULTILINE)
        result = result.replace('\n', '')
        result = result.replace('\t', '')
        result = result.replace('\xa0', '')
        comments_clean.append(result)
        time.sleep(random.uniform(3.2,5.4))
    return comments_clean

In [7]:
# x - оценка, y - страница 
database = {}
votes = [1,5]
for x in votes:
    for y in tqdm(range(1,4)):
        url = 'https://www.banki.ru/services/responses/bank/sberbank/?rate='+ str(x) +'&page='+ str(y) + '&isMobile=0'
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        d = writing_in_dict(soup)
        a = str(x)+str(y)
        database[a] = d 

100%|██████████| 3/3 [05:25<00:00, 108.38s/it]


In [8]:
database['11'].extend(database['12'])
database['51'].extend(database['52'])
database['13'].extend(database['53'])

In [35]:
control_dict = {}
pos_list = []
neg_list = []
for each in database['13']:
    if each != '':
        if each in database['53']:
            pos_list.append(each)
        else:
            neg_list.append(each)
control_dict['pos'] = pos_list
control_dict['neg'] = neg_list

In [10]:
neg_coments = database['11']
pos_coments = database['51']
control_group = database['13']

In [11]:
neg_comments_30 = random.sample(neg_coments, 30)
pos_comments_30 = random.sample(pos_coments, 30)
control = random.sample(control_group, 10)

In [12]:
str_pos = ''.join(pos_comments_30)
str_neg = ''.join(neg_comments_30)
str_control = ''.join(control)

In [13]:
words_pos = [w.lower() for w in word_tokenize(str_pos) if w.isalpha()]
words_neg = [w.lower() for w in word_tokenize(str_neg) if w.isalpha()]
sw = stopwords.words('russian')
filtered_pos = [w for w in words_pos if w not in sw]
filtered_neg = [w for w in words_neg if w not in sw]

morph = MorphAnalyzer()
lemmas_pos = []
for word in filtered_pos:
    lemmas_pos.append(morph.parse(word)[0].normal_form)
lemmas_neg = []
for word in filtered_neg:
    lemmas_neg.append(morph.parse(word)[0].normal_form)

In [14]:
cnt_pos = Counter()
for word in lemmas_pos:
    cnt_pos[word] += 1
cnt_neg = Counter()
for word in lemmas_neg:
    cnt_neg[word] += 1

In [15]:
d_pos = {}
d_neg = {}
for key in dict(cnt_pos):
    if key not in dict(cnt_neg).keys() and cnt_pos[key]>1:
        d_pos[key] = cnt_pos[key]
for key in dict(cnt_neg):
    if key not in dict(cnt_pos).keys() and cnt_neg[key]>1:
        d_neg[key] = cnt_neg[key]

In [16]:
freq_lists = {}
freq_lists['pos'] = d_pos
freq_lists['neg'] = d_neg

In [39]:
def detect_comment(freq_lists, text):
    counts = Counter()
    for emotion, freq_list in freq_lists.items():
        freq_list = Counter(freq_list)
        words = [w.lower() for w in word_tokenize(text) if w.isalpha()]
        filtered = [w for w in words if w not in sw]
        lemmas = []
        for word in filtered:
            lemmas.append(morph.parse(word)[0].normal_form)
        for word in lemmas:
            counts[emotion] += int(freq_list[word] > 0)
    return counts.most_common()

In [20]:
test_texts = random.choice(control_group)
print(test_texts)

Хочу выразить благодарность сотрудникам Сбербанка Виноградовой Татьяне Викторовне и Потаповой Оксане Владимировне, за их чуткий подход и заботу о клиентах. Обратилась с запутанным вопросом по поводу наследства и выплаты средств, как можно скорее, в итоге помогли быстро и понятно все разрешить. Прошу руководства обратить внимание на профессионализм данных сотрудников и примеривать по итогам месяца.Теперь по всем банковским вопросам будем с семьей обращаться в ваше отделение.


In [21]:
detect_comment(freq_lists, test_texts)

[('pos', 7), ('neg', 1)]

In [37]:
def test_detect_comment(freq_lists, test_size):
    results = []  # результаты
    gold = []     # исходная эмоция
    for emotion in ['pos', 'neg']:
        for text in control_dict[emotion]:
            predicted_emotion = detect_comment(freq_lists, text)[0][0]
            results.append(predicted_emotion)
            gold.append(emotion)
    print("RESULTS:")
    print("%d languages" % 2)
    print("Test size: %d texts per emotion" % test_size)
    print("Accuracy: %.4f" % accuracy_score(results, gold))

In [40]:
test_detect_comment(freq_lists, 10)

RESULTS:
2 languages
Test size: 10 texts per emotion
Accuracy: 0.8163


# Оптимизация программы:
1. Запись сразу после Counter'а в словарь
2. Сделать одинаковыми по размеру словари тональности (негативных слов получилось больше)
3. Выкачать больше отзывов за больший промежуток времени, так как встречались отзывы, которые повторяли ситуацию (например, первый негативный отзыв про банк, второй отзыв с такой же ситуацией, но уже с другими деталями, условно, банк обещал сделать X и Y, но все еще ничего не сделал; аналогично с противоположной ситуацией)